# Basic Langchain Features

This notebook covers some of the fundamental Features of Langchain.

In [33]:
import os
import credentials

#set environment variable for OpenAI API Key
os.environ["OPENAI_API_KEY"] = credentials.llm_api_key

## Call a LLM Model

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5)
text = "What are 5 vacation destinations in sweden for someboy who likes nature?"
print(llm(text))



1. Abisko National Park
2. Koster Islands
3. Gotland
4. Jokkmokk
5. Lulea Archipelago


## Prompt Tempplates
LLMs take promps as inputs. Typically this is not simply a hardcoded string but rather a combination of a template, some examples, and user input. 

A prompt template refers to a reproducible way to generate a prompt. It contains a text string ("the template"), that can take in a set of parameters from the end user and generates a prompt.

A prompt template can contain:

* instructions to the language model,
* a set of few shot examples to help the language model generate a better response,
* a question to the language model.

In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["sport"],
    template="What are 5 vacation destinations for someone who likes {sport}?",
)

In [8]:
print(prompt.format(sport="basketball"))

What are 5 vacation destinations for someone who likes basketball?


In [10]:
print(llm(prompt.format(sport="basketball")))



1. New York City, USA - Home of the NBA's New York Knicks and Brooklyn Nets.
2. Los Angeles, USA - Home of the NBA's Los Angeles Lakers and Los Angeles Clippers.
3. Chicago, USA - Home of the NBA's Chicago Bulls.
4. Toronto, Canada - Home of the NBA's Toronto Raptors.
5. Las Vegas, USA - Host of the NBA Summer League.


## Chains: Combine multiple LLMs and prompts in multi-step workflows
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

We define a Chain very generically as a sequence of calls to components, which can include other chains. 

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.5)

prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 5 vacation destinations for someone who likes to eat {food}?",
)

In [11]:
#chain the LLM and the prompt. We are not manually feeding the prompt into the language model
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("fruit"))



1. Costa Rica - known for its abundance of tropical fruits like pineapple, coconut, papaya, and mango.

2. Hawaii - known for its delicious tropical fruits like pineapple, guava, lychee, and papaya.

3. Thailand - known for its exotic fruits like durian, mangosteen, lychee, and rambutan.

4. Jamaica - known for its sweet and juicy fruits like bananas, mangoes, papayas, and oranges.

5. India - known for its unique and flavorful fruits like lychee, guava, jackfruit, and mango.


Let's try chaining the chat module in a LLM chain.

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate)

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.5)

#chain chat and prompt together
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

ColorfulStride


## Agents: Dynamically call chains based on user input

Some applications require a flexible chain of calls to LLMs and other tools based on user input. The Agent interface provides the flexibility for such applications. An agent has access to a suite of tools, and determines which ones to use depending on the user input. Agents can use multiple tools, and use the output of one tool as the input to the next.

There are two main types of agents:

* Action agents: at each timestep, decide on the next action using the outputs of all previous actions
* Plan-and-execute agents: decide on the full sequence of actions up front, then execute them all without updating the plan

Action agents are suitable for small tasks, while plan-and-execute agents are better for complex or long-running tasks that require maintaining long-term objectives and focus. Often the best approach is to combine the dynamism of an action agent with the planning abilities of a plan-and-execute agent by letting the plan-and-execute agent use action agents to execute plans.

https://python.langchain.com/docs/modules/agents/

In [28]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# Load the model
llm = OpenAI(temperature=0)

In [29]:
#Load tools for the agent to use
#load serpapi api key. Serpapi let's us scrape google searches
os.environ["SERPAPI_API_KEY"] = credentials.serpapi_api_key
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [30]:
# Let's initialize an agent with:
# 1. The tools
# 2. The language model
# 3. The type of agent we want to use.

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [6]:
#Run agent
agent.run("Who is the current leader of Sweden? What is the largest prime number that is smaller than their age?")



> Entering new  chain...
 I need to find out who the leader of Sweden is and then calculate the largest prime number that is smaller than their age.
Action: Search
Action Input: "leader of Sweden"
Observation: Ulf Kristersson
Thought: I need to find out the age of Ulf Kristersson
Action: Search
Action Input: "Ulf Kristersson age"
Observation: 59 years
Thought: I need to calculate the largest prime number that is smaller than 59
Action: Calculator
Action Input: 59
Observation: Answer: 59
Thought: I now know the final answer
Final Answer: Ulf Kristersson is the current leader of Sweden and the largest prime number that is smaller than their age is 59.

> Finished chain.


'Ulf Kristersson is the current leader of Sweden and the largest prime number that is smaller than their age is 59.'

Let's try another math based prompt.

In [11]:
#Run agent
agent.run("What is the average age of joe biden and donald trump?")



> Entering new  chain...
 I need to find out the ages of both Joe Biden and Donald Trump.
Action: Search
Action Input: "Joe Biden age"
Observation: 80 years
Thought: I need to find out Donald Trump's age
Action: Search
Action Input: "Donald Trump age"
Observation: 77 years
Thought: I now know the ages of both Joe Biden and Donald Trump
Action: Calculator
Action Input: (80 + 77) / 2
Observation: Answer: 78.5
Thought: I now know the final answer
Final Answer: The average age of Joe Biden and Donald Trump is 78.5 years.

> Finished chain.


'The average age of Joe Biden and Donald Trump is 78.5 years.'

In [31]:
#Run agent
agent.run("How many guitar stores are there in Chicago?")



> Entering new  chain...
 I need to find out how many guitar stores are in Chicago.
Action: Search
Action Input: "guitar stores in Chicago"
Observation: https://www.yelp.com/search?cflt=guitarstores&find_loc=Chicago%2C+IL
Thought: I can use this list to count the number of stores.
Action: Count
Action Input: Number of stores on the list
Observation: Count is not a valid tool, try another one.
Thought: I can use the search engine to count the number of stores.
Action: Search
Action Input: "number of guitar stores in Chicago"
Observation: https://www.yelp.com/search?cflt=guitarstores&find_loc=Chicago%2C+IL
Thought: I can use this search result to count the number of stores.
Action: Count
Action Input: Number of stores on the list
Observation: Count is not a valid tool, try another one.
Thought: I can use the search engine to find the answer.
Action: Search
Action Input: "number of guitar stores in Chicago"
Observation: https://www.yelp.com/search?cflt=guitarstores&find_loc=Chicago%2C+I

'Agent stopped due to iteration limit or time limit.'

### Agent that searches csv file

Let's try to querry a college basketball statistics csv file.

https://www.kaggle.com/datasets/andrewsundberg/college-basketball-dataset

In [34]:
from langchain.agents import create_csv_agent

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [35]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    "cbb.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [25]:
agent.run("Can you provide the descriptive statistics for my data? Is there any data missing?")



> Entering new  chain...
Thought: I need to get the descriptive statistics for the data and check for missing data.
Action: python_repl_ast
Action Input: df.describe()
Observation:                  G            W        ADJOE        ADJDE      BARTHAG  \
count  2455.000000  2455.000000  2455.000000  2455.000000  2455.000000   
mean     31.492464    16.284318   103.304481   103.304603     0.493957   
std       2.657401     6.610960     7.376981     6.605318     0.256244   
min      15.000000     0.000000    76.600000    84.000000     0.005000   
25%      30.000000    11.000000    98.300000    98.500000     0.282200   
50%      31.000000    16.000000   103.000000   103.500000     0.475000   
75%      33.000000    21.000000   108.000000   107.900000     0.712200   
max      40.000000    38.000000   129.100000   124.000000     0.984200   

             EFG_O        EFG_D          TOR         TORD          ORB  ...  \
count  2455.000000  2455.000000  2455.000000  2455.000000  2455.000000 

'There is no missing data in the dataframe. The descriptive statistics for the dataframe are provided in the observation above.'

In [41]:
agent.run("What team had the most offensive rebounds (ORB) 2015 to 2019?")



> Entering new  chain...
Thought: I need to find the team with the highest ORB value
Action: python_repl_ast
Action Input: df[df['YEAR'] >= 2015].sort_values(by='ORB', ascending=False).head(1)
Observation:            TEAM  CONF   G   W  ADJOE  ADJDE  BARTHAG  EFG_O  EFG_D   TOR  ...  \
682  Quinnipiac  MAAC  30  15  101.4  101.3   0.5029   44.0   45.2  20.4  ...   

     FTRD  2P_O  2P_D  3P_O  3P_D  ADJ_T   WAB  POSTSEASON  SEED  YEAR  
682  33.4  43.1  41.9  30.7  35.1   66.7 -10.9         NaN   NaN  2015  

[1 rows x 24 columns]
Thought: I now know the final answer
Final Answer: Quinnipiac had the most offensive rebounds from 2015 to 2019.

> Finished chain.


'Quinnipiac had the most offensive rebounds from 2015 to 2019.'

## Memory - send conversation history to agents

In [12]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

In [13]:
conversation.predict(input="What's up? My name is Erik.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What's up? My name is Erik.
AI:

> Finished chain.


" Hi Erik! My name is AI. It's nice to meet you. I'm doing great. How about you?"

In [14]:
conversation.predict(input="I'm doing well! Can you tell me how many guitar stores there are in Chicago?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's up? My name is Erik.
AI:  Hi Erik! My name is AI. It's nice to meet you. I'm doing great. How about you?
Human: I'm doing well! Can you tell me how many guitar stores there are in Chicago?
AI:

> Finished chain.


" I'm sorry, Erik, I don't know the answer to that question."

In [15]:
conversation.predict(input="What was the first thing I said to you?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's up? My name is Erik.
AI:  Hi Erik! My name is AI. It's nice to meet you. I'm doing great. How about you?
Human: I'm doing well! Can you tell me how many guitar stores there are in Chicago?
AI:  I'm sorry, Erik, I don't know the answer to that question.
Human: What was the first thing I said to you?
AI:

> Finished chain.


' You said, "What\'s up? My name is Erik."'